In [227]:
import numpy as np
import math
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
import json
#
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#
import xgboost as xgb

In [228]:
df = pd.read_csv("data/df_12379.csv")
df.head()

,HeatID,HeatOrderID,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,L10,L11,L12,L13,L14,L15,L16,L17,L18,L19
0,16383,18802,3,4,5197,5036,1020,6528,1516,4859,...,0,2,0,0,1,0,1,0,0,0
1,16380,18853,28,10,4985,4995,720,5856,1606,4763,...,0,1,0,0,5,0,4,0,0,0
2,16357,18800,3,39,6682,6556,610,6435,1603,5460,...,0,2,0,0,8,0,0,9,0,0
3,16356,18799,3,53,6707,6572,690,6493,1607,5184,...,3,4,0,0,10,0,0,16,0,0
4,16354,18797,3,27,6117,6908,770,6481,1516,4740,...,2,3,0,0,1,0,0,0,0,0


In [229]:
# remove unneccesary columns
columns = df.columns.tolist()
remove_columns = ['HeatID', 'HeatOrderID','Energy']
features_columns = [x for x in columns if x not in remove_columns]

In [230]:
# train ANN
X = df[features_columns].values
y = df[['Energy']].values
epochs = 10

In [231]:
def build_and_compile_ann():
    model = keras.Sequential([
      layers.Dense(50, activation='sigmoid'), 
      layers.Dense(100, activation='sigmoid'),
      layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

def train_ann(X_train, y_train, epochs):
    ann_model = build_and_compile_ann()
    history = ann_model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        verbose=0, epochs=epochs)
    return ann_model, history

def train_xgb(X_train, y_train):
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
    xgb_model.fit(X_train, y_train)
    return xgb_model

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# xgboost
xgb_model = train_xgb(X_train, y_train)
# ann
ann_model, history = train_ann(X_train, y_train, epochs=epochs)

# testing 
y_pred_ann = ann_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)
r2_ann = r2_score(y_test, y_pred_ann.flatten())
r2_xgb = r2_score(y_test, y_pred_xgb)
print("R2 score of ANN ", r2_ann)
print("R2 score of XGBoost ", r2_xgb)
#
metadata = {'ANN_R2': r2_ann, 'XGB_R2': r2_xgb, 'epochs': epochs}
#
ann_model.save(f'models/ANN')
xgb_model.save_model(f'models/XGB.json')

with open('models/metadata.json', 'w') as file:
    json.dump(metadata, file)

print("Training complete")

5/5 [==============================] - 0s 33ms/step
R2 score of ANN  -406.91074584214425
R2 score of XGBoost  0.5673882214305788


INFO:tensorflow:Assets written to: models/ANN/assets


INFO:tensorflow:Assets written to: models/ANN/assets


Training complete


In [9]:
np.random.RandomState(1234)
np.random.seed(42)

# chemical component to achieve: C 1,55 / Si 0,4 / Mn 0,3 / Cr 11,8 / Mo 0,75 / V 0,82
chemi_component = (np.array([1.55,0.4,0.3,11.8,0.75,0.82]) / 100.0).astype(np.float32)

chemi_names = ['C','Si','Mn','Cr','Mo','V']

# total weight for the final steel production
total_weight = 10_000

total_chemi_to_achieve = total_weight * chemi_component

df_chemi = pd.read_excel("data/chemi.xlsx", engine="openpyxl")
df_price = pd.read_excel("data/scrap.xlsx", engine="openpyxl")

# function to return all information from df
def df_columns_name(df):
    columns = df.columns.tolist()
    remove_columns = ['HeatID', 'HeatOrderID','Energy']
    features_columns = [x for x in columns if x not in remove_columns]
    
    # constant process parameter, start with "Feature"
    constant_features_names = [x for x in features_columns if "Feature" in x]
    
    #"K" means Kreislauf, "L" means Legierungen, "F" means Fremdschrotte, we only want to optimize "F"
    schrotte_features_names = [x for x in features_columns if "Feature" not in x]
    
    # schrotte name for Kreislauf
    kreislauf_schrotte_names = [x for x in features_columns if "K" in x]
    
    # schrotte name for legierung
    legierung_schrotte_names = [x for x in features_columns if "L" in x]
    
    # schrotte name for Fremdschrotte
    fremd_schrotte_names = [x for x in features_columns if "F" in x and len(x) < 4]
    
    return constant_features_names, schrotte_features_names, kreislauf_schrotte_names,legierung_schrotte_names,fremd_schrotte_names

# function to calculate the chemical component weights
# randomly choose a row from df

constant_features_names, schrotte_features_names, kreislauf_schrotte_names, legierung_schrotte_names,fremd_schrotte_names = df_columns_name(df)

def calculate_chemi_component(df, df_chemi):
    
    """
    return the randomly chosen row, and its constant column, kreislauf column and 
    legierung column, use them to return the chemical component of fremdschrotte column
    """
    df_random_row = df.sample()
    
    # calculate the constant features
    constant_column = (df_random_row[constant_features_names].values[0]).astype(np.float32)
    
    # calculate the chemical component for kreislauf
    kreislauf_column = (df_random_row[kreislauf_schrotte_names].values[0]).astype(np.float32)
    kreislauf_chemical_table = df_chemi[chemi_names].iloc[len(kreislauf_schrotte_names)-1:]
    chemi_component_kreislauf = (np.dot(kreislauf_column, kreislauf_chemical_table) /100.0).astype(np.float32)
    
    # calculate the chemical component for legierungen
    legierung_column = (df_random_row[legierung_schrotte_names].values[0]).astype(np.float32)
    legierung_chemical_table = df_chemi[chemi_names].iloc[len(fremd_schrotte_names):len(kreislauf_schrotte_names)-1]
    chemi_component_legierung = (np.dot(legierung_column, legierung_chemical_table) /100.0).astype(np.float32)
    
    # calculate the chemical compoent for fremdschrotte
    chemi_to_achieve_fremdschrotte = (np.abs(total_chemi_to_achieve - chemi_component_kreislauf - chemi_component_legierung)).astype(np.float32)
    
    return constant_column, kreislauf_column, legierung_column, chemi_to_achieve_fremdschrotte

# construct the chemical table
# assume that every company's chemical elements for every schrott is identical
def fremdschrott_chemi_table(df_chemi):
    df_chemi_fremdschrott= df_chemi[chemi_names].iloc[:len(fremd_schrotte_names)]
    fremdschrott_chemi = df_chemi_fremdschrott.T
    n_times = 9
    temp_dfs = []
    
    for col_name in fremdschrott_chemi.columns:
        temp_df = fremdschrott_chemi[[col_name]].copy()
        for i in range(1, n_times+1):
            temp_df[f'{col_name}{i}'] = fremdschrott_chemi[col_name]
        temp_dfs.append(temp_df)

    fremdschrotte_chemi_table = pd.concat(temp_dfs, axis=1) / 100.0
    
    return fremdschrotte_chemi_table

length_fremdschrott = len(fremd_schrotte_names)
length_company = 10

total_variable_length = length_fremdschrott * length_company

df_price_list = df_price["price"].to_numpy()
price_list = df_price_list[:total_variable_length]


x_lower = np.zeros(total_variable_length)
x_upper = np.ones(total_variable_length) * total_weight


fremdschrotte_chemi_table = fremdschrott_chemi_table(df_chemi)
# right hand side of equality constraints
aeq = np.array(fremdschrotte_chemi_table)

# start values
x_start = np.zeros(total_variable_length)

# scipy specific
#bounds = Bounds([0.0]*total_variable_length, [max(total_chemi_to_achieve)]*total_variable_length)

# max iteration
max_iter = 50 #1000

In [240]:
# function to calculate the logistic cost of tf version
@tf.function
def sum_t3_tf(x):
    """
    X: tf.Tensor, the quantity of every scrap
    """
    summe = tf.constant(0.0, dtype=tf.float32)
    quantity = tf.reduce_sum(tf.reshape(x,[-1,10]), axis=1) #tf.convert_to_tensor([sum(g) for g in list(grouper(10, x))])

    for q in quantity:
        q = tf.reshape(q, ())
        summe += tf.cond(q <= 10.0, 
                         lambda: 0.0, 
                         lambda: tf.cond(q > 10.0 and q <= 50.0, 
                                         lambda: 2.5 * (q - 10), 
                                         lambda: 100.0))
    return summe

@tf.function
def tf_ann(x,constant_column,kreislauf_column,legierung_column):
    # x = tf.convert_to_tensor(x, dtype=tf.float32)

    list_fremdschrotte = tf.reduce_sum(tf.reshape(x,[-1,10]), axis=1)
    x = tf.concat([constant_column, kreislauf_column, list_fremdschrotte, legierung_column], axis=0)
    x = tf.expand_dims(x, axis=0)
    ann_pred = ann_model(x)
    return ann_pred

# function for calculate the total cost, tf version
@tf.function
def objective_tf(x,constant_column,kreislauf_column,legierung_column):
    #x = tf.convert_to_tensor(x, dtype=tf.float32)
    price_list_tf = tf.convert_to_tensor(price_list, dtype=tf.float32)
    t1 = tf.tensordot(x, price_list_tf,axes=1)
    t2 = tf_ann(x,constant_column,kreislauf_column,legierung_column)
    t3 = sum_t3_tf(x) 
    
    #return t1 +t2 + t3
    return t2

In [241]:
@tf.function
def grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column):
    x1 = tf.convert_to_tensor(x, dtype=tf.float32)
    constant_column = tf.convert_to_tensor(constant_column, dtype=tf.float32)
    kreislauf_column = tf.convert_to_tensor(kreislauf_column, dtype=tf.float32)
    legierung_column = tf.convert_to_tensor(legierung_column, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x1)
        t2 = objective_tf(x1,constant_column,kreislauf_column,legierung_column)
        y = tape.jacobian(t2, x1)
    return y

@tf.function
def hess_grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column):
    x1 = tf.convert_to_tensor(x, dtype=tf.float32)
    constant_column = tf.convert_to_tensor(constant_column, dtype=tf.float32)
    kreislauf_column = tf.convert_to_tensor(kreislauf_column, dtype=tf.float32)
    legierung_column = tf.convert_to_tensor(legierung_column, dtype=tf.float32)
            
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x1)
        obj_fun = objective_tf(x1,constant_column,kreislauf_column,legierung_column)
        y = tape.gradient(obj_fun, x1)
        h = tape.jacobian(y, x1)
    
    return y, h

In [242]:
x = np.random.randn(160).astype(np.float32)

constant_column, kreislauf_column, legierung_column, \
    chemi_to_achieve_fremdschrotte = calculate_chemi_component(df, df_chemi)
    

In [243]:
tf_ann(x,constant_column,kreislauf_column,legierung_column)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[31.699856]], dtype=float32)>

In [244]:
objective_tf(x,constant_column,kreislauf_column,legierung_column)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[31.699856]], dtype=float32)>

In [ ]:
output = grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column)
output

In [239]:
out2 = hess_grad_f_ann_tf(x,constant_column,kreislauf_column,legierung_column)
out2[0]

ERROR:tensorflow:Got error while pfor was converting op name: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf"
op: "StatelessIf"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/Reshape_grad/Reshape_grad/Reshape"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_3_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_4_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_5_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/g

ERROR:tensorflow:Got error while pfor was converting op name: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf"
op: "StatelessIf"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/Reshape_grad/Reshape_grad/Reshape"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_3_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_4_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_5_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/g

ERROR:tensorflow:name: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf"
op: "StatelessIf"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/Reshape_grad/Reshape_grad/Reshape"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_3_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_4_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_5_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_gr

ERROR:tensorflow:name: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf"
op: "StatelessIf"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_grad/StatelessIf/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/Reshape_grad/Reshape_grad/Reshape"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_3_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_4_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/TensorListPushBack_5_grad/TensorListPopBack:1"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/gradients/while_grad/gradients/while/cond_grad/StatelessIf_gr

ERROR:tensorflow:Got error while pfor was converting op name: "gradients/gradients/while_grad/while_grad_grad/gradients/while_grad/while_grad_grad"
op: "StatelessWhile"
input: "gradients/gradients/while_grad/while_grad_grad/grad_counter"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/while_grad/while_grad_grad/gradients/while_grad/while_grad/while/maximum_iterations"
input: "gradients/gradients/while_grad/while_grad_grad/gradients/while_grad/while_grad_grad/gradients/while_grad/while_grad"
input: "gradients/grad_ys_1"
input: "gradients/gradients/TensorArrayUnstack/TensorListFromTensor_grad/TensorListStack_grad/TensorListFromTensor"
input: "gradients/grad_ys_2"
input: "gradients/grad_ys_3"
input: "gradients/grad_ys_4"
input: "gradients/grad_ys_5"
input: "gradients/grad_ys_6"
input: "gradients/grad_ys_7"
input: "gradients/grad_ys_8"
input: "gradients/grad_ys_9"
input: "gradients/grad_ys_10"
input: "gradients/grad_ys_11"
input: "gradients/grad_ys_12"
input: "gradients/gr

...

ERROR:tensorflow:Got error while pfor was converting op name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "loop_body/zeros_4"
input: "loop_body/zeros_5"
input: "loop_body/zeros_6"
input: "loop_body/zeros_7"
input: "loop_body/zeros_8"
input: "loop_body/zeros_9"
input: "loop_body/zeros_10"
input: "loop_body/zeros_11"
input: "loop_body/zeros_12"
input: "loop_body/zeros_13"
input: "loop_body/zeros_14"
input: "loop_body/zeros_15"
input: "loop_body/zeros_17"
input: "loop_body/zeros_like"
input: "loop_body/zeros_like_1"
input: "loop_body/zeros_like_2"
input: "loop_body/zeros_like_3"
input: "loop_body/zeros_like_6"
input: "loop_body/zeros_like_7"
input: "loop_body/zeros_like_8"
input: "loop_body/zeros_like_9"
input: "loop_body/zeros_like_10"
input: "loop_body/zeros_like_11"
input: "loop_body/zeros_like_12"
input: "loop_body/zeros_20"
input: "loop

ERROR:tensorflow:Got error while pfor was converting op name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "loop_body/zeros_4"
input: "loop_body/zeros_5"
input: "loop_body/zeros_6"
input: "loop_body/zeros_7"
input: "loop_body/zeros_8"
input: "loop_body/zeros_9"
input: "loop_body/zeros_10"
input: "loop_body/zeros_11"
input: "loop_body/zeros_12"
input: "loop_body/zeros_13"
input: "loop_body/zeros_14"
input: "loop_body/zeros_15"
input: "loop_body/zeros_17"
input: "loop_body/zeros_like"
input: "loop_body/zeros_like_1"
input: "loop_body/zeros_like_2"
input: "loop_body/zeros_like_3"
input: "loop_body/zeros_like_6"
input: "loop_body/zeros_like_7"
input: "loop_body/zeros_like_8"
input: "loop_body/zeros_like_9"
input: "loop_body/zeros_like_10"
input: "loop_body/zeros_like_11"
input: "loop_body/zeros_like_12"
input: "loop_body/zeros_20"
input: "loop

ERROR:tensorflow:name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "loop_body/zeros_4"
input: "loop_body/zeros_5"
input: "loop_body/zeros_6"
input: "loop_body/zeros_7"
input: "loop_body/zeros_8"
input: "loop_body/zeros_9"
input: "loop_body/zeros_10"
input: "loop_body/zeros_11"
input: "loop_body/zeros_12"
input: "loop_body/zeros_13"
input: "loop_body/zeros_14"
input: "loop_body/zeros_15"
input: "loop_body/zeros_17"
input: "loop_body/zeros_like"
input: "loop_body/zeros_like_1"
input: "loop_body/zeros_like_2"
input: "loop_body/zeros_like_3"
input: "loop_body/zeros_like_6"
input: "loop_body/zeros_like_7"
input: "loop_body/zeros_like_8"
input: "loop_body/zeros_like_9"
input: "loop_body/zeros_like_10"
input: "loop_body/zeros_like_11"
input: "loop_body/zeros_like_12"
input: "loop_body/zeros_20"
input: "loop_body/zeros_21"
input: "PartitionedCall

ERROR:tensorflow:name: "loop_body/PartitionedCall"
op: "PartitionedCall"
input: "gradient_tape/Reshape"
input: "loop_body/zeros"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "loop_body/zeros_4"
input: "loop_body/zeros_5"
input: "loop_body/zeros_6"
input: "loop_body/zeros_7"
input: "loop_body/zeros_8"
input: "loop_body/zeros_9"
input: "loop_body/zeros_10"
input: "loop_body/zeros_11"
input: "loop_body/zeros_12"
input: "loop_body/zeros_13"
input: "loop_body/zeros_14"
input: "loop_body/zeros_15"
input: "loop_body/zeros_17"
input: "loop_body/zeros_like"
input: "loop_body/zeros_like_1"
input: "loop_body/zeros_like_2"
input: "loop_body/zeros_like_3"
input: "loop_body/zeros_like_6"
input: "loop_body/zeros_like_7"
input: "loop_body/zeros_like_8"
input: "loop_body/zeros_like_9"
input: "loop_body/zeros_like_10"
input: "loop_body/zeros_like_11"
input: "loop_body/zeros_like_12"
input: "loop_body/zeros_20"
input: "loop_body/zeros_21"
input: "PartitionedCall

ValueError: in user code:

    File "/tmp/ipykernel_3585/3938761199.py", line 24, in hess_grad_f_ann_tf  *
        h = tape.jacobian(y, x1)

    ValueError: Encountered an exception while vectorizing the jacobian computation. Vectorization can be disabled by setting experimental_use_pfor to False.


In [224]:
out2[1]

<tf.Tensor: shape=(160, 160), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>